In [1]:
!pip install tensorboardX
!pip install wandb


[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import os
os.environ['WANDB_PROJECT']='finetuning'

In [3]:
device = "cuda"
#config = PeftConfig.from_pretrained(peft_model_id)
#model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path)
#tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
#model.resize_token_embeddings(len(tokenizer))
#model = PeftModel.from_pretrained(model, peft_model_id)

In [4]:
!pip install trl
!pip install datasets
!pip install peft
import os
from enum import Enum
from functools import partial
import pandas as pd
import torch

from transformers import AutoModelForCausalLM, LlamaTokenizer, TrainingArguments, set_seed
from datasets import load_dataset
from trl import SFTTrainer
from peft import get_peft_model, LoraConfig, TaskType

seed = 42
set_seed(seed)


[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [5]:
os.environ['WANDB_PROJECT']='finetuning'


In [6]:
from huggingface_hub import notebook_login
notebook_login()

In [7]:
model_name = "mistralai/Mistral-7B-v0.1"
dataset_name = "FinGPT/fingpt-sentiment-train"
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained(model_name)
dataset = load_dataset(dataset_name)

In [9]:
template = '''###Instruction-> {instruction} ***
###Input-> {input} ***
###Response-> {output} ***'''

In [10]:
def preprocess(data):
    processed_data = []
    # print(data)
    for instruction_text, input_text, output_text in zip(data['instruction'], data['input'], data['output']):
        processed_prompt = template.format(input=input_text,instruction=instruction_text,output=output_text)
        #tokenizer.pad_token = tokenizer.eos_token
        #tokenized_inputs = tokenizer(processed_prompt,return_tensors="pt",padding=True,max_length=4096,truncation=False)
        processed_data.append(processed_prompt)
    return {"content": processed_data}



In [11]:
processed_data = dataset.map(preprocess,batched=True,remove_columns=dataset["train"].column_names)
#processed_data=processed_data.remove_columns(['instruction','input','output'])
processed_data = processed_data['train'].train_test_split(0.1)


In [12]:
processed_data

DatasetDict({
    train: Dataset({
        features: ['content'],
        num_rows: 69094
    })
    test: Dataset({
        features: ['content'],
        num_rows: 7678
    })
})

In [13]:
print(processed_data)
print(processed_data["train"]["content"][0])


DatasetDict({
    train: Dataset({
        features: ['content'],
        num_rows: 69094
    })
    test: Dataset({
        features: ['content'],
        num_rows: 7678
    })
})
###Instruction-> What is the sentiment of this tweet? Please choose an answer from {negative/neutral/positive}. ***
###Input-> Heading to Tokyo this week? Explore our city guide, including a royal 5k run, to-die-for tempura and readers' tips… https://t.co/eLapFuHIih ***
###Response-> neutral ***


In [14]:

peft_config = LoraConfig(r=8,
                         lora_alpha=16,
                         lora_dropout=0.1,
                         target_modules=["gate_proj","q_proj","lm_head","o_proj","k_proj","embed_tokens","down_proj","up_proj","v_proj"],
                         task_type=TaskType.CAUSAL_LM)

In [15]:

class ChatmlSpecialTokens(str, Enum):
    
    assistant = "###Response"
    user = "###Input"
    system = "###Instruction"
    eos_token = "***"
    bos_token = "->"
    pad_token = "<pad>"

    @classmethod
    def list(cls):
        return [c.value for c in cls]

tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        pad_token=ChatmlSpecialTokens.pad_token.value,
        bos_token=ChatmlSpecialTokens.bos_token.value,
        eos_token=ChatmlSpecialTokens.eos_token.value,
        additional_special_tokens=ChatmlSpecialTokens.list(),
        trust_remote_code=True
    )
tokenizer.chat_template = template
model = AutoModelForCausalLM.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# cast non-trainable params in fp16
for p in model.parameters():
    if not p.requires_grad:
        p.data = p.to(torch.float16)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


trainable params: 21,549,120 || all params: 7,263,313,984 || trainable%: 0.2967


In [16]:
output_dir = "Mistral-7B-v0.1"
per_device_train_batch_size = 2
per_device_eval_batch_size = 2
gradient_accumulation_steps = 8
logging_steps = 20
learning_rate = 5e-4
max_grad_norm = 1.0
max_steps = 125
num_train_epochs=4
warmup_ratio = 0.1
lr_scheduler_type = "cosine"
max_seq_length = 2048

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    save_strategy="no",
    evaluation_strategy="epoch",
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    max_grad_norm=max_grad_norm,
    weight_decay=0.1,
    warmup_ratio=warmup_ratio,
    lr_scheduler_type=lr_scheduler_type,
    fp16=True,
    report_to=["tensorboard", "wandb"],
    hub_private_repo=True,
    push_to_hub=True,
    num_train_epochs=num_train_epochs,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False}
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [17]:
trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=processed_data['train'],
    eval_dataset=processed_data['test'],
    tokenizer=tokenizer,
    packing=True,
    dataset_text_field='content',
    max_seq_length=max_seq_length,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': packing, dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:195: UserWarning: You passed a `packing` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will overri

In [68]:
processed_data["train"]

Dataset({
    features: ['content'],
    num_rows: 69094
})

In [ ]:
processed_data['train']['content']

In [18]:
trainer.train()
trainer.save_model()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: shriramjay1 (shriramjay1-gmail-create). Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Epoch,Training Loss,Validation Loss
0,6.759000,0.762342
1,4.709400,0.593684
2,3.766500,0.519142
3,3.125800,0.504949


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:227: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


events.out.tfevents.1730193619.7a7e4cd6a865:   0%|          | 0.00/4.53k [00:00<?, ?B/s]

...

events.out.tfevents.1730194162.08dfa6cb464d:   0%|          | 0.00/6.20k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/611M [00:00<?, ?B/s]

events.out.tfevents.1730208736.08dfa6cb464d:   0%|          | 0.00/10.8k [00:00<?, ?B/s]

events.out.tfevents.1730209653.08dfa6cb464d:   0%|          | 0.00/13.3k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.56k [00:00<?, ?B/s]

In [21]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
import torch

peft_model_id = "workspace/finetuned_sentiment_mistral7b"
device = "cuda"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
model.resize_token_embeddings(len(tokenizer))
model = PeftModel.from_pretrained(model, peft_model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [54]:
print(processed_data["train"][0]['content'])

###Instruction-> What is the sentiment of this tweet? Please choose an answer from {negative/neutral/positive}. ***
###Input-> Heading to Tokyo this week? Explore our city guide, including a royal 5k run, to-die-for tempura and readers' tips… https://t.co/eLapFuHIih ***
###Response-> neutral ***


In [19]:
trainer.save_model('workspace/finetuned_sentiment_mistral7b')

No files have been modified since last commit. Skipping to prevent empty commit.


In [25]:
# Diye gaye sentence ko English mein translate kare:\n  Maine 10 baar customer care ko call kiya hai par abhi tak mujhe mera order nahi mila hai.
# bro, ye generative AI kya hai?
# भारत में पर्यटन के बारे में एक निबंध लिखें।

model.to(torch.float16)
model.cuda()
model.eval()
messages = [
    {"role": "system", "content": "What is the sentiment of this tweet?",
    "role": "user", "content": "What happened in "},
]
text = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
inputs = tokenizer(text, return_tensors="pt")#, add_special_tokens=False)
inputs = {k: v.to("cuda") for k,v in inputs.items()}
outputs = model.generate(**inputs, 
                         max_new_tokens=128, 
                         do_sample=True, 
                         top_p=0.95, 
                         temperature=0.2, 
                         repetition_penalty=1.1, 
                         eos_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0]))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


->###Instruction-> {instruction} ***
###Input-> {input} ***
###Response-> {output} ***
###Response-> neutral ***
